In [48]:
import os
import sys
import math
import logging
from pathlib import Path
import json
import glob
import re

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import sagemaker
import boto3
from IPython.display import HTML
import time
from time import gmtime, strftime
#from misc import get_execution_role, wait_for_s3_object
from sagemaker.pytorch import PyTorch, PyTorchModel
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))
aws_region = sage_session.boto_region_name
print("AWS region: {}".format(aws_region))

S3 bucket path: s3://sagemaker-us-west-2-351033349862/
AWS region: us-west-2


**PLEASE** save this file right now using the following naming convention: `NUMBER_FOR_SORTING-YOUR_INITIALS-SHORT_DESCRIPTION`, e.g. `1.0-fw-initial-data-exploration`. Use the number to order the file within the directory according to its usage.

In [50]:
# run in local mode?
local_mode = True

if local_mode:
    instance_type = 'local'
else:
    instance_type = "ml.c5.2xlarge"

In [51]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::351033349862:role/service-role/AmazonSageMaker-ExecutionRole-20200227T101932


In [52]:
financial_data_path = Path('../data/preprocessed/stock_contexts_rewards.csv').resolve()
financial_data_path_str = financial_data_path.as_posix()
financial_data_path_str

'/home/ec2-user/SageMaker/space_bandits_deployment/data/preprocessed/stock_contexts_rewards.csv'

In [53]:
financial_data_s3_path = sage_session.upload_data(path=financial_data_path_str, bucket=s3_bucket)

In [54]:
source_dir = Path('../src/space_bandits_deployment').resolve().as_posix()
train_script = Path('../src/space_bandits_deployment/train.py').resolve().as_posix()

In [55]:
hyperparameters = {"action_dim": 8, "context_dim":21}

In [56]:
estimator = PyTorch(entry_point="train.py",source_dir=source_dir,role=role,
                        train_instance_type=instance_type,

                    train_instance_count=1,
                        output_path=s3_output_path,framework_version='1.3.1',hyperparameters=hyperparameters)

In [57]:
estimator.fit(inputs={'train': financial_data_s3_path})

INFO:sagemaker:Creating training-job with name: pytorch-training-2020-03-05-03-06-36-011
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-dmk7t:
    command: train
    environment:
    - AWS_REGION=us-west-2
    - TRAINING_JOB_NAME=pytorch-training-2020-03-05-03-06-36-011
    image: 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.3.1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-dmk7t
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmpu411gj83/algo-1-dmk7t/output:/opt/ml/output
    - /tmp/tmpu411gj83/algo-1-dmk7t/output/data:/opt/ml/output/data
    - /tmp/tmpu411gj83/algo-1-dmk7t/input:/opt/ml/input
    - /tmp/tmpu411gj83/model:/opt/ml/model
    - /tmp/tmpb66zspe_:/opt/ml/input/data/train
version: '2.3'

INFO:sagemaker.local.image:docke

In [11]:
sage_session = sagemaker.local.LocalSession() if local_mode else sage_session


In [58]:
job_name = estimator.latest_training_job.job_name
job_name

'pytorch-training-2020-03-05-03-06-36-011'

In [59]:
if local_mode:
    output_path_prefix = f"{job_name}/output.tar.gz"
    model_path = f"{job_name}/model.tar.gz"
    
else:
    output_path_prefix = f"{job_name}/output/output.tar.gz"
    model_path = f"{job_name}/output/model.tar.gz"
    
#sage_session.download_data(path="../models/output", bucket=s3_bucket, key_prefix=job_name)

In [60]:
model_path = os.path.join(s3_output_path,model_path)
model_path

's3://sagemaker-us-west-2-351033349862/pytorch-training-2020-03-05-03-06-36-011/model.tar.gz'

In [61]:
instance_type = "ml.m4.xlarge"

In [99]:
model=PyTorchModel(model_data=model_path, name="model-v1",
    role=role, framework_version='1.3.1', source_dir=source_dir,entry_point='serve.py')

In [100]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: model-v1
INFO:sagemaker:Creating endpoint with name model-v1
-------------------!

In [204]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type,)


INFO:sagemaker:Creating model with name: pytorch-training-2020-03-04-20-41-49-033
INFO:sagemaker:Creating endpoint with name pytorch-training-2020-03-04-20-41-49-033
INFO:sagemaker.local.image:serving
INFO:sagemaker.local.image:creating hosting dir in /tmp/tmpqn6715a1
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-dcoms:
    command: serve
    environment:
    - SAGEMAKER_PROGRAM=train.py
    - SAGEMAKER_SUBMIT_DIRECTORY=s3://sagemaker-us-west-2-351033349862/pytorch-training-2020-03-04-20-41-49-033/model.tar.gz
    - SAGEMAKER_ENABLE_CLOUDWATCH_METRICS=false
    - SAGEMAKER_CONTAINER_LOG_LEVEL=20
    - SAGEMAKER_REGION=us-west-2
    image: 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:1.3.1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-dcoms
  

Exception in thread Thread-17:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 614, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 673, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 619, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpqn6715a1/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited wi

In [101]:
df = pd.read_csv(financial_data_path)

In [102]:
data = df.to_numpy()

In [103]:
data[0].shape

(29,)

In [109]:
np.argmax(data[0,21:])

1

In [104]:
context = data[0,:21]
context

array([-0.424286,  0.880005,  0.5     ,  0.289997,  0.82    ,  0.17    ,
        0.370003,  0.07    ,  0.630001, -0.830001,  0.32    ,  0.119999,
        0.24    ,  0.200001, -0.550003, -0.075   ,  0.059997, -0.130001,
        0.610001,  0.059997,  0.470002])

In [107]:
predictor.content_type

'application/x-npy'

In [26]:
predictor.content_type="application/x-npy"

In [92]:
from six import BytesIO
def _npy_dumps(data):
    """
    Serialized a numpy array into a stream of npy-formatted bytes.
    """
    buffer = BytesIO()
    np.save(buffer, data)
    return buffer.getvalue()

In [93]:
context_bytes = _npy_dumps(context)

In [108]:
context_bytes, context

(b"\x93NUMPY\x01\x00v\x00{'descr': '<f8', 'fortran_order': False, 'shape': (21,), }                                                           \n\x80\xa4\x89w\x80'\xdb\xbf\x00 \xea>\x00)\xec?\x00\x00\x00\x00\x00\x00\xe0?\x00\n\xbc\x93O\x8f\xd2?\x00\n\xd7\xa3p=\xea?\x80\xf5(\\\x8f\xc2\xc5?\x01\x00\x1b\x10!\xae\xd7?\x00Q\xb8\x1e\x85\xeb\xb1?@Pn\xdb\xf7(\xe4?\x00\xb7\xd4A^\x8f\xea\xbf\x80\x14\xaeG\xe1z\xd4?\xff{\x8d$A\xb8\xbe?\xff\x1e\x85\xebQ\xb8\xce?\x80j\x15\xfd\xa1\x99\xc9?\x00v6\xe4\x9f\x99\xe1\xbf\x0023333\xb3\xbf\x00X\xb7A\xed\xb7\xae?\x01A\xb9m\xdf\xa3\xc0\xbf\xc0\xdf0\xd1 \x85\xe3?\x00X\xb7A\xed\xb7\xae?\x00~\xc3D\x83\x14\xde?",
 array([-0.424286,  0.880005,  0.5     ,  0.289997,  0.82    ,  0.17    ,
         0.370003,  0.07    ,  0.630001, -0.830001,  0.32    ,  0.119999,
         0.24    ,  0.200001, -0.550003, -0.075   ,  0.059997, -0.130001,
         0.610001,  0.059997,  0.470002]))

In [106]:
predictor.predict(context)

array([1])

In [110]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: model-v1
INFO:sagemaker:Deleting endpoint with name: model-v1


In [111]:
predictor.delete_model()

INFO:sagemaker:Deleting model with name: model-v1
